<a href="https://colab.research.google.com/github/fancagi/intro_pulp/blob/main/Gestion_de_operaciones_L4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problemas de programación de personal

Muchas clínicas funcionan las 24 horas del día, durante los 7 días de la semana. El número de empleados necesarios puede variar de una hora a otra. 

Consideremos un contexto en el cual los empleados están asignados a un turno de 8 o 4 horas. Los posibles horarios de inicio de turno y la duración se presenta en la siguiente tabla . Cualquier persona que trabaje en el turno de noche recibirá un pago más alto por ese turno. El problema es determinar cuántos empleados asignar a cada uno de los turnos para cumplir con los requisitos de mano de obra por hora al mínimo costo posible.

Existen, de esta forma, 7 diferentes tipos de turnos cada uno con unas demandas diferentes de personal. 

| Horarios de inicio| Duración del turno  | Pago  |
|--------------------|--------------------|---------------|
| 6 | 8  horas | 10 |
| 12 | 8  horas | 10 |
| 16 | 8  horas | 10 |
| 20 | 8  horas | 20 |
| 22 | 8  horas | 20 |
| 6 | 4  horas | 6 |
| 10 | 4  horas | 6 |
| 14 | 4  horas | 6 |
| 20 | 4  horas | 12 |


In [ ]:
# Requerimientos por hora

import numpy as np

dem=[4, 7, 6, 5, 6, 6, 6, 3, 7, 7, 6, 6, 5, 5, 3, 4, 5, 4, 6, 5, 6,2, 6, 4]
start_time=[6,12,16,20,22,6,10,14,20]
duration=[8,8,8,8,8,4,4,4,4]
wage=[10,10,10,20,20,6,6,6,12]

params=[]
# print as markdown table
for s in range(len(start_time)):
    params.append([start_time[s],duration[s],wage[s]])

params=np.array(params)    

params

array([[ 6,  8, 10],
       [12,  8, 10],
       [16,  8, 10],
       [20,  8, 20],
       [22,  8, 20],
       [ 6,  4,  6],
       [10,  4,  6],
       [14,  4,  6],
       [20,  4, 12]])

In [ ]:
contribution_shift_hour=np.zeros((len(dem),len(start_time)))
for i in start_time:
    for j in range(duration[start_time.index(i)]):
        contribution_shift_hour[(i+j)%24,start_time.index(i)]=1
            
contribution_shift_hour            

array([[0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 1., 0., 0., 0., 0., 1., 0., 0.],
       [1., 1., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0.

In [ ]:
# Sets

shifts=set(range(len(start_time)))
# set of shifts that work during hour h
M=[set() for h in range(len(dem))]
for h in range(len(dem)):
    M[h].union({s for s in shifts if contribution_shift_hour[h,s]==1})

In [ ]:
import pulp as plp

# Create the 'prob' variable to contain the problem data
prob = plp.LpProblem("The problem", plp.LpMinimize)

x=plp.LpVariable.dicts("x", (shifts), lowBound=0, cat='Integer')

prob += plp.lpSum([wage[s]*x[s] for s in shifts])

for h in range(len(dem)):
    prob += plp.lpSum([contribution_shift_hour[h,s]*x[s] for s in shifts]) >= dem[h], "Demand"+str(h)

prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/user/opt/anaconda3/envs/apricot-env/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/88fcae2ea1834deca5e7d6b72e67c609-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/88fcae2ea1834deca5e7d6b72e67c609-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 29 COLUMNS
At line 105 RHS
At line 130 BOUNDS
At line 140 ENDATA
Problem MODEL has 24 rows, 9 columns and 48 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 288 - 0.00 seconds
Cgl0003I 0 fixed, 5 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 5 rows, 5 columns (5 integer (0 of which binary)) and 11 elements
Cutoff increment increased from 1e-05 to 1.9999
Cbc0012I Integer solution of 288

/Users/user/opt/anaconda3/envs/apricot-env/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [ ]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

x_0 = 7.0
x_1 = 1.0
x_2 = 5.0
x_3 = 1.0
x_4 = 6.0
x_5 = 0.0
x_6 = 0.0
x_7 = 3.0
x_8 = 0.0
